# comercio_ext_indices.tb_referencia_ncm_cgce
> ### Origem — `bronze/autoloader/landingbeca2026jan/balancacomercial/NCM_CGCE_delta`
## 📌 Descrição do arquivo

Classificação **CGCE** em estrutura hierárquica (N1 → N2 → N3), com descrições em **PT/EN/ES**.

|Coluna|Descrição técnica|Interpretação humana|
|---|---|---|
|`CO_CGCE_N3`|Código CGCE nível 3|Categoria econômica detalhada|
|`NO_CGCE_N3`|Nome CGCE N3 (PT)|Nome da categoria (PT)|
|`NO_CGCE_N3_ING`|Nome CGCE N3 (EN)|Nome da categoria (EN)|
|`NO_CGCE_N3_ESP`|Nome CGCE N3 (ES)|Nome da categoria (ES)|
|`CO_CGCE_N2`|Código CGCE nível 2|Categoria intermediária|
|`NO_CGCE_N2`|Nome CGCE N2 (PT)|Nome do grupo (PT)|
|`NO_CGCE_N2_ING`|Nome CGCE N2 (EN)|Nome do grupo (EN)|
|`NO_CGCE_N2_ESP`|Nome CGCE N2 (ES)|Nome do grupo (ES)|
|`CO_CGCE_N1`|Código CGCE nível 1|Macro-categoria|
|`NO_CGCE_N1`|Nome CGCE N1 (PT)|Macro-categoria (PT)|
|`NO_CGCE_N1_ING`|Nome CGCE N1 (EN)|Macro-categoria (EN)|
|`NO_CGCE_N1_ESP`|Nome CGCE N1 (ES)|Macro-categoria (ES)|

## Configurações
> #### **imports**
> #### **get files**
> #### **schema**

In [0]:
import org . apache . spark  . sql . functions . _
import org . apache . spark  . sql . types . _
import org . apache . spark  . sql . DataFrame
import org . apache . hadoop . fs  . FileSystem 
import org . apache . hadoop . fs  . Path 
import io  . delta  . tables . _

In [0]:

val bronzePath =     "/mnt/bronze/autoloader/landingbeca2026jan/balancacomercial/NCM_CGCE_delta/"   
val silverPath =     "/mnt/silver/landingbeca2026jan/comercio_ext_indices/tb_referencia_ncm_cgce/"   
val silverTable =    "tb_referencia_ncm_cgce"
val fs = FileSystem.get(spark.sparkContext.hadoopConfiguration)



In [0]:

val silverSchema = StructType(
    Seq (
      StructField ( "CO_CGCE_N3" , StringType ,  nullable = false ) ,
      StructField ( "NO_CGCE_N3" , StringType ,  nullable = false ) ,
      StructField ( "NO_CGCE_N3_ING" , StringType ,  nullable = false ) ,
      StructField ( "NO_CGCE_N3_ESP" , StringType ,  nullable = false ) ,
      StructField ( "CO_CGCE_N2" , StringType ,  nullable = false ) ,
      StructField ( "NO_CGCE_N2" , StringType ,  nullable = false ) ,
      StructField ( "NO_CGCE_N2_ING" , StringType ,  nullable = false ) ,
      StructField ( "NO_CGCE_N2_ESP" , StringType ,  nullable = false ) ,
      StructField ( "CO_CGCE_N1" , StringType ,  nullable = false ) ,
      StructField ( "NO_CGCE_N1" , StringType ,  nullable = false ) ,
      StructField ( "NO_CGCE_N1_ING" , StringType ,  nullable = false ) ,
      StructField ( "NO_CGCE_N1_ESP" , StringType ,  nullable = false ) ,
      StructField ( "TS_REF" , TimestampType ,  nullable = false ) ,
      StructField ( "NM_ORIGEM" , StringType ,  nullable = false ) 
    )
  )

## Extração
> #### **saprk.read**

In [0]:
val dfBronzeRaw = spark . read . format ( "delta" ) . load ( bronzePath )

##Normalização
> #### **datatype**
> #### **regras**

In [0]:
val dfNormalized = dfBronzeRaw
  .withColumn("CO_CGCE_N3", trim(upper(col("CO_CGCE_N3").cast(StringType))))
  .withColumn("NO_CGCE_N3", col("NO_CGCE_N3").cast(StringType))
  .withColumn("NO_CGCE_N3_ING", col("NO_CGCE_N3_ING").cast(StringType))
  .withColumn("NO_CGCE_N3_ESP", col("NO_CGCE_N3_ESP").cast(StringType))
  .withColumn("CO_CGCE_N2", trim(upper(col("CO_CGCE_N2").cast(StringType))))
  .withColumn("NO_CGCE_N2", col("NO_CGCE_N2").cast(StringType))
  .withColumn("NO_CGCE_N2_ING", col("NO_CGCE_N2_ING").cast(StringType))
  .withColumn("NO_CGCE_N2_ESP", col("NO_CGCE_N2_ESP").cast(StringType))
  .withColumn("CO_CGCE_N1", trim(upper(col("CO_CGCE_N1").cast(StringType))))
  .withColumn("NO_CGCE_N1", col("NO_CGCE_N1").cast(StringType))
  .withColumn("NO_CGCE_N1_ING", col("NO_CGCE_N1_ING").cast(StringType))
  .withColumn("NO_CGCE_N1_ESP", col("NO_CGCE_N1_ESP").cast(StringType))

In [0]:
val dfWithDefaults = dfNormalized
  .withColumn("TS_REF", current_timestamp())
  .withColumn("NM_ORIGEM", lit("/landingbeca2026jan/balancacomercial/NCM_CGCE_delta"))

##Validações
> #### **data quality**
> #### **deduplicação**
> #### **schema fit**

In [0]:

val dfValid = dfWithDefaults . filter ( 
    col("CO_CGCE_N3").isNotNull
  )


In [0]:
val dfDedup = dfValid . dropDuplicates (
    Seq (
        "CO_CGCE_N3"
    )
  )

In [0]:
val dfSilver = dfDedup.select( col("CO_CGCE_N3"), col("NO_CGCE_N3"), col("NO_CGCE_N3_ING"), col("NO_CGCE_N3_ESP"), col("CO_CGCE_N2"), col("NO_CGCE_N2"), col("NO_CGCE_N2_ING"), col("NO_CGCE_N2_ESP"), col("CO_CGCE_N1"), col("NO_CGCE_N1"), col("NO_CGCE_N1_ING"), col("NO_CGCE_N1_ESP"), col("TS_REF"), col("NM_ORIGEM") )

##Carga
> #### **overwrite**

In [0]:
spark.conf.set("spark.databricks.optimizer.dynamicPartitionPruning", "true")
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")

val silverExists = fs.exists(new Path(silverPath))

if (!silverExists) {
  dfSilver
    .write
    .format("delta")
    .mode("overwrite")
    .save(silverPath)
}